Retreive Amazon review data.

TODO: Save large files to disk and train model against them in batches.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#!pip install anaconda
!pip install transformers
"""
# https://stackoverflow.com/a/59331412/9295513
# same
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
# update 1
!conda install -q -y --prefix /usr/local python=3.6 ujson
# update 2
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
"""
#!conda env create --file "/content/drive/My Drive/Colab-Notebooks/environment.yml"


"\n# https://stackoverflow.com/a/59331412/9295513\n# same\n!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh\n!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh\n!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local\n# update 1\n!conda install -q -y --prefix /usr/local python=3.6 ujson\n# update 2\nimport sys\nsys.path.append('/usr/local/lib/python3.6/site-packages')\n"

In [ ]:
#https://github.com/bhargaviparanjape/clickbait
DATA_URL = 'https://github.com/bhargaviparanjape/clickbait/raw/master/dataset/clickbait_data.gz'
SAMPLE_SIZE = 60000

MODEL_SAVE_PATH = '/data/user/jprob/bert2gpt'#'/content/drive/My Drive/Colab-Notebooks/bert2gpt'
MODEL_TEMP_PATH = '/data/user/jprob/temp'#'/content/drive/My Drive/Colab-Notebooks/temp'

In [ ]:
import os
import gzip
from urllib.request import urlopen
from tqdm.auto import tqdm

try:
  os.mkdir("training_data")
except FileExistsError:
  pass # The directory already exists.

def get_data(url):
    data = []
    with gzip.open(urlopen(url)) as f:
        for l in f:
          if l != b'\n':
            data.append(str(l)[2:].rstrip(r"\n'"))
    return data

u_input = input("First time?[y] for yes, anything else for no")

if u_input.lower() == "y":
  data = get_data(DATA_URL)

  # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
  n = 20
  data_batches = [data[i:i + n] for i in range(0, len(data), n)]

  for c, i in enumerate(data_batches):
    with open(f"training_data/{c}.txt", "w") as f:
      f.write(str(i))
else:
  data = []
  for i in os.listdir("training_data"):
    with open("training_data/" + i, "r") as f:
      f.readline()
      for l in f:
        data.append(json.loads(l.strip()))

Create product ASIN mapping to link products and their reviews.

In [ ]:
import re

# Remove HTML tags from text
def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)
"""
product_dataset = {}
for product in metadata:
    asin = product['asin']
    category = ' '.join(product['category'])
    title = product['title']
    description = ' '.join(product['description'])
    
    # Remove HTML tags
    filtered_description = remove_html_tags(description)
    
    product_dataset[asin] = {
        'category': category,
        'title': title,
        'description': filtered_description
    }"""
  

"\nproduct_dataset = {}\nfor product in metadata:\n    asin = product['asin']\n    category = ' '.join(product['category'])\n    title = product['title']\n    description = ' '.join(product['description'])\n    \n    # Remove HTML tags\n    filtered_description = remove_html_tags(description)\n    \n    product_dataset[asin] = {\n        'category': category,\n        'title': title,\n        'description': filtered_description\n    }"

Initialize tokenizer and model.

In [ ]:
from transformers import BertTokenizer, GPT2Tokenizer, EncoderDecoderModel

RATING_TOKEN = '[RAT]'
CATEGORY_TOKEN = '[CAT]'
TITLE_TOKEN = '[TTL]'
DESCRIPTION_TOKEN = '[DES]'
LABEL_MASK_TOKEN_ID = -100

ENCODER_MAX_LENGTH = 512
DECODER_MAX_LENGTH = 128

encoder_tokenizer = BertTokenizer(
    '/content/drive/My Drive/Colab-Notebooks/vocab.txt'#,
    #additional_special_tokens=[RATING_TOKEN,CATEGORY_TOKEN,TITLE_TOKEN,DESCRIPTION_TOKEN,],
)
encoder_tokenizer.bos_token = encoder_tokenizer.cls_token
encoder_tokenizer.eos_token = encoder_tokenizer.sep_token

def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs

GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens
decoder_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
decoder_tokenizer.pad_token = decoder_tokenizer.unk_token

#https://huggingface.co/transformers/model_doc/bert.html below number is default vocab size, vocab_size=30522
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-cased', 'gpt2')
model.config.vocab_size = 30522 - 1 #model.config.encoder.vocab_size #should be vocab_size - 1, find the vocab size; vocab size of model should be same as tokenizer

model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.eos_token_id = decoder_tokenizer.eos_token_id
model.config.pad_token_id = decoder_tokenizer.eos_token_id

model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.crossattention.bias', 'h.0.crossattention.masked_bias', 'h.0.crossattention.c_attn.weight', 'h.0.crossattention.c_attn.bias', 'h.0.crossattention.q_attn.weight', 'h.0.crossattention.q_attn.bias', 'h.0.crossattention.c_proj.weight', 'h.0.crossattention.c_proj.bias', 'h.0.ln_cross_attn.weight', 'h.0.ln_cross_attn.bias', 'h.1.crossattention.bias', 'h.1.crossattention.masked_bias', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.1.crossattention.q_attn.weight', 'h.1.crossattention.q_attn.bias', 'h.1.crossattention.c_proj.weight', 'h.1.crossattention.c_proj.bias', 'h.1.ln_cross_attn.weight', 'h.1.ln_cross_attn.bias', 'h.2.crossattention.bias', 'h.2.crossattention.masked_bias', 'h.2.crossattention.c_attn.weight', 'h.2.crossattention.c_attn.bias', 'h.2.crossattention.q_attn.weight', 'h.2.crossattention.q_attn.bias', 'h.2.crossattention.c_proj.weight'

Map data to model inputs.

In [ ]:
# map data to input
def generate_input_sequence(rating, category, title, description):
    rating_sequence = '{} {}'.format(RATING_TOKEN, rating)
    category_sequence = '{} {}'.format(CATEGORY_TOKEN, category)
    title_sequence = '{} {}'.format(TITLE_TOKEN, title)
    description_sequence = '{} {}'.format(DESCRIPTION_TOKEN, description)
    return ' '.join([rating_sequence, category_sequence, title_sequence, description_sequence])

# tokenize input for encoder
def preprocess_encoder_input(sequence):
    tokenized_input = encoder_tokenizer.tokenize(sequence)
    encoded_input = encoder_tokenizer.convert_tokens_to_ids(tokenized_input)
    prepared_input = encoder_tokenizer.prepare_for_model(
        encoded_input,
        max_length=ENCODER_MAX_LENGTH,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
    )
    return prepared_input['input_ids'], prepared_input['attention_mask']

# tokenize input for decoder
def preprocess_decoder_input(sequence):
    tokenized_input = decoder_tokenizer.tokenize(sequence)
    encoded_input = decoder_tokenizer.convert_tokens_to_ids(tokenized_input)
    prepared_input = decoder_tokenizer.prepare_for_model(
        encoded_input,
        max_length=DECODER_MAX_LENGTH,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
    )
    return prepared_input['input_ids'], prepared_input['attention_mask']

model_inputs = []
for review in tqdm(data):
    #product_asin = review['asin']
    #review_rating = review['overall']
    #review_text = review.get('reviewText')
    
    #product = product_dataset.get(product_asin)
    
    # filter reviews with missing information
    has_text_review = True #review_text is not None
    product_exists = True #product is not None
    product_has_description = True #product_exists and product['description'] is not []
    

    if has_text_review and product_has_description:
        
        product_combined = generate_input_sequence(
            " ", #review_rating,
            " ", #product['category'],
            " ", #product['title'],
            " ", #product['description'],
        )
        
        input_ids, attention_mask = preprocess_encoder_input(product_combined)
        decoder_input_ids, decoder_attention_mask = preprocess_decoder_input(review) #review_text
        
        # mask pad tokens in label
        labels = []
        for index in decoder_input_ids:
            label = index
            if index is decoder_tokenizer.pad_token_id:
                label = LABEL_MASK_TOKEN_ID
            labels.append(label)
    
        model_inputs.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'decoder_input_ids': decoder_input_ids,
            'decoder_attention_mask': decoder_attention_mask,
            'labels': labels,
        })

Split into training and testing inputs. Initialize datasets.

In [ ]:
import sys
!{sys.executable} -mpip install datasets==1.0.2

In [ ]:
from datasets import Dataset
import pandas as pd

data_frame = pd.DataFrame.from_records(model_inputs)
dataset = Dataset.from_pandas(data_frame).select(range(SAMPLE_SIZE))
dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)
dataset_dict = dataset.train_test_split()
train_dataset = dataset_dict['train']
test_dataset = dataset_dict['test']

Install seq2seq trainer prerequisites

In [ ]:
# !pip install rouge_score
!{sys.executable} -mpip install rouge_score
# !rm seq2seq_trainer.py
# !rm seq2seq_training_args.py
# !wget https://raw.githubusercontent.com/huggingface/transformers/v3.5.1/examples/seq2seq/seq2seq_trainer.py
# !wget https://raw.githubusercontent.com/huggingface/transformers/v3.5.1/examples/seq2seq/seq2seq_training_args.py

Train model.

In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")

def compute_metrics(outputs):
    predictions_ids = outputs.predictions
    labels_ids = outputs.label_ids

    predictions = decoder_tokenizer.batch_decode(predictions_ids, skip_special_tokens=True)
    labels_ids[labels_ids == LABEL_MASK_TOKEN_ID] = decoder_tokenizer.eos_token_id
    labels = decoder_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=predictions,
        references=labels,
        rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

The next cell is taken from the transformers examples directory. Importing was broken for it and it alone for some reason, so I copied it and removed things until it worked.

In [ ]:
from typing import Any, Dict, List, Optional, Tuple, Union

import torch
from torch import nn
from torch.utils.data import DistributedSampler, RandomSampler

from transformers import PreTrainedModel, Trainer, logging
from transformers.file_utils import is_torch_tpu_available
from transformers.models.fsmt.configuration_fsmt import FSMTConfig
from transformers.optimization import (
    Adafactor,
    AdamW,
    get_constant_schedule,
    get_constant_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup,
)
from transformers.trainer_pt_utils import get_tpu_sampler
#from transformers.training_args import ParallelMode


logger = logging.get_logger(__name__)

arg_to_scheduler = {
    "linear": get_linear_schedule_with_warmup,
    "cosine": get_cosine_schedule_with_warmup,
    "cosine_w_restarts": get_cosine_with_hard_restarts_schedule_with_warmup,
    "polynomial": get_polynomial_decay_schedule_with_warmup,
    "constant": get_constant_schedule,
    "constant_w_warmup": get_constant_schedule_with_warmup,
}


class Seq2SeqTrainer(Trainer):
    def __init__(self, config=None, data_args=None, *args, **kwargs):
        super().__init__(*args, **kwargs)

        if config is None:
            assert isinstance(
                self.model, PreTrainedModel
            ), f"If no `config` is passed the model to be trained has to be of type `PreTrainedModel`, but is {self.model.__class__}"
            self.config = self._actual_model(self.model).config
        else:
            self.config = config

        self.data_args = data_args
        self.vocab_size = self.config.tgt_vocab_size if isinstance(self.config, FSMTConfig) else self.config.vocab_size

        if self.args.label_smoothing != 0 or (self.data_args is not None and self.data_args.ignore_pad_token_for_loss):
            assert (
                self.config.pad_token_id is not None
            ), "Make sure that `config.pad_token_id` is correcly defined when ignoring `pad_token` for loss calculation or doing label smoothing."

        if self.config.pad_token_id is None and self.config.eos_token_id is not None:
            logger.warn(
                f"The `config.pad_token_id` is `None`. Using `config.eos_token_id` = {self.config.eos_token_id} for padding.."
            )

        if self.args.label_smoothing == 0:
            self.loss_fn = torch.nn.CrossEntropyLoss(ignore_index=self.config.pad_token_id)
        else:
            # dynamically import label_smoothed_nll_loss
            from utils import label_smoothed_nll_loss

            self.loss_fn = label_smoothed_nll_loss

    def create_optimizer_and_scheduler(self, num_training_steps: int):
        """
        Setup the optimizer and the learning rate scheduler.
        We provide a reasonable default that works well. If you want to use something else, you can pass a tuple in the
        Trainer's init through :obj:`optimizers`, or subclass and override this method in a subclass.
        """
        if self.optimizer is None:
            no_decay = ["bias", "LayerNorm.weight"]
            optimizer_grouped_parameters = [
                {
                    "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                    "weight_decay": self.args.weight_decay,
                },
                {
                    "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                    "weight_decay": 0.0,
                },
            ]
            if self.args.adafactor:
                self.optimizer = Adafactor(
                    optimizer_grouped_parameters,
                    lr=self.args.learning_rate,
                    scale_parameter=False,
                    relative_step=False,
                )

            else:
                self.optimizer = AdamW(
                    optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon
                )

        if self.lr_scheduler is None:
            self.lr_scheduler = self._get_lr_scheduler(num_training_steps)
        else:  # ignoring --lr_scheduler
            logger.warn("scheduler is passed to `Seq2SeqTrainer`, `--lr_scheduler` arg is ignored.")

    def _get_lr_scheduler(self, num_training_steps):
        schedule_func = arg_to_scheduler[self.args.lr_scheduler]
        if self.args.lr_scheduler == "constant":
            scheduler = schedule_func(self.optimizer)
        elif self.args.lr_scheduler == "constant_w_warmup":
            scheduler = schedule_func(self.optimizer, num_warmup_steps=self.args.warmup_steps)
        else:
            scheduler = schedule_func(
                self.optimizer, num_warmup_steps=self.args.warmup_steps, num_training_steps=num_training_steps
            )
        return scheduler

    def _get_train_sampler(self) -> Optional[torch.utils.data.sampler.Sampler]:
        if isinstance(self.train_dataset, torch.utils.data.IterableDataset):
            return None
        elif is_torch_tpu_available():
            return get_tpu_sampler(self.train_dataset)
        else:
            if self.args.sortish_sampler:
                self.train_dataset.make_sortish_sampler(
                    self.args.per_device_train_batch_size,
                    distributed=(self.args.parallel_mode == ParallelMode.DISTRIBUTED),
                )

            return (
                RandomSampler(self.train_dataset)
                if self.args.local_rank == -1
                else DistributedSampler(self.train_dataset)
            )

    def _compute_loss(self, model, inputs, labels):
        if self.args.label_smoothing == 0:
            if self.data_args is not None and self.data_args.ignore_pad_token_for_loss:
                # force training to ignore pad token
                logits = model(**inputs, use_cache=False)[0]
                loss = self.loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
            else:
                # compute usual loss via models
                loss, logits = model(**inputs, labels=labels, use_cache=False)[:2]
        else:
            # compute label smoothed loss
            logits = model(**inputs, use_cache=False)[0]
            lprobs = torch.nn.functional.log_softmax(logits, dim=-1)
            loss, _ = self.loss_fn(lprobs, labels, self.args.label_smoothing, ignore_index=self.config.pad_token_id)
        return loss, logits

    def compute_loss(self, model, inputs):
        labels = inputs.pop("labels")
        loss, _ = self._compute_loss(model, inputs, labels)
        return loss

    def prediction_step(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
    ) -> Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
        """
        Perform an evaluation step on :obj:`model` using obj:`inputs`.
        Subclass and override to inject custom behavior.
        Args:
            model (:obj:`nn.Module`):
                The model to evaluate.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.
            prediction_loss_only (:obj:`bool`):
                Whether or not to return the loss only.
        Return:
            Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
            A tuple with the loss, logits and labels (each being optional).
        """
        inputs = self._prepare_inputs(inputs)

        gen_kwargs = {
            "max_length": self.data_args.val_max_target_length
            if self.data_args is not None
            else self.config.max_length,
            "num_beams": self.data_args.eval_beams if self.data_args is not None else self.config.num_beams,
        }

        if self.args.predict_with_generate and not self.args.prediction_loss_only:
            generated_tokens = self.model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                **gen_kwargs,
            )
            # in case the batch is shorter than max length, the output should be padded
            if generated_tokens.shape[-1] < gen_kwargs["max_length"]:
                generated_tokens = self._pad_tensors_to_max_len(generated_tokens, gen_kwargs["max_length"])

        labels = inputs.pop("labels")
        with torch.no_grad():
            # compute loss on predict data
            loss, logits = self._compute_loss(model, inputs, labels)

        loss = loss.mean().detach()
        if self.args.prediction_loss_only:
            return (loss, None, None)

        logits = generated_tokens if self.args.predict_with_generate else logits

        if labels.shape[-1] < gen_kwargs["max_length"]:
            labels = self._pad_tensors_to_max_len(labels, gen_kwargs["max_length"])

        return (loss, logits, labels)

    def _pad_tensors_to_max_len(self, tensor, max_length):
        # If PAD token is not defined at least EOS token has to be defined
        pad_token_id = self.config.pad_token_id if self.config.pad_token_id is not None else self.config.eos_token_id

        if pad_token_id is None:
            raise ValueError(
                f"Make sure that either `config.pad_token_id` or `config.eos_token_id` is defined if tensor has to be padded to `max_length`={max_length}"
            )

        padded_tensor = pad_token_id * torch.ones(
            (tensor.shape[0], max_length), dtype=tensor.dtype, device=tensor.device
        )
        padded_tensor[:, : tensor.shape[-1]] = tensor
        return padded_tensor
import logging
from dataclasses import dataclass, field
from typing import Optional

#from seq2seq_trainer import arg_to_scheduler
from transformers import TrainingArguments


logger = logging.getLogger(__name__)


@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    """
    Parameters:
        label_smoothing (:obj:`float`, `optional`, defaults to 0):
            The label smoothing epsilon to apply (if not zero).
        sortish_sampler (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether to SortishSamler or not. It sorts the inputs according to lenghts in-order to minimizing the padding size.
        predict_with_generate (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether to use generate to calculate generative metrics (ROUGE, BLEU).
    """

    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear",
        metadata={"help": f"Which lr scheduler to use. Selected in {sorted(arg_to_scheduler.keys())}"},
    )

In [ ]:
%debug

> /usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py(880)<genexpr>()
    878     @staticmethod
    879     def _unnest(py_dict):
--> 880         return dict((key, array[0]) for key, array in py_dict.items())
    881 
    882     @staticmethod

ipdb> print(py_dict)
*** NameError: name 'py_dict' is not defined
ipdb> print py_dict
*** SyntaxError: Missing parentheses in call to 'print'. Did you mean print(py_dict)?
ipdb> print(py_dict)
*** NameError: name 'py_dict' is not defined


In [ ]:
BATCH_SIZE = 4
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    output_dir=os.path.join(MODEL_TEMP_PATH, 'results'),
    overwrite_output_dir=True,
    #save_steps=10,
    save_steps=500,
    evaluation_strategy='steps',
    #eval_steps=4,
    eval_steps=7500,
    logging_dir=os.path.join(MODEL_TEMP_PATH, 'runs'),
    #logging_steps=2,
    logging_steps=1000,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    fp16=True,
    warmup_steps=2000,
    save_total_limit=3,
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

IndexError: ignored

Save model to disk.

In [ ]:
model.save_pretrained(MODEL_SAVE_PATH)

Load model from disk

In [ ]:
model = EncoderDecoderModel.from_pretrained(MODEL_SAVE_PATH)

Predict from input

In [ ]:
"""
import torch

product = list(product_dataset.values())[0]
product_category = product['category']
product_title = product['title']
product_description = product['description']
product_combined = generate_input_sequence(5, product_category, product_title, product_description)
"""

input_ids, _ = preprocess_encoder_input(product_combined)
input_ids = torch.tensor(input_ids).unsqueeze(0)

output_ids = model.generate(
    input_ids,
    decoder_start_token_id=model.config.decoder.pad_token_id,
    temperature=1.3,
    top_k=9,
    top_p=0.9,
    repetition_penalty=1.4
)

In [ ]:
for i in output_ids[:10]:
  print(i)